# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Kacem Khaled
    - Oumayma Messoussi
    - Semah Aissaoui




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [1]:
import numpy as np
import math
from math import ceil
import copy
from collections import deque
from random import randint
import random

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        self.rock = None

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = self.rock
        return s

    def put_rock(self, rock_pos):
        s = copy.deepcopy(self)
        s.prev = self
        s.nb_moves = self.nb_moves + 1
        s.rock = rock_pos
        return s
    
    """
    estimee1: distance entre la voiture rouge et la sortie
    """
    def estimee1(self):
        return 4 - self.pos[0]
    """
    estimee2: estimee1  + score de pondération de nombre de voitures qui bloquent les voitures qui bloquent ,etc... 
    avec une profondeur déterminée (exemple: 5) dans 'rh.blocking_cars_level_general(self,5)' et les coûts de 
    libération de leurs chemins (le niveau le plus profond a la pondération la plus faible)
    """
    def estimee2(self, rh):
        list_blockers, move_away_score,blockers_counting_score = rh.blocking_cars_level_general(self,5)
        return self.estimee1()+move_away_score + blockers_counting_score
        
    """
    estimation de nombre de mouvements nécessaires pour de la libération du chemin direct 
    de la voiture 'blocked' bloquée par une voiture 'blocker'
    """
    
    def move_blocker_away_cost(self,rh,blocked,blocker):
        move_away_cost = 0
        move_away_cost_up = move_away_cost_down = 0
        if rh.move_on[blocked] - rh.length[blocker] >= 0:        #can we move it up / left?
            move_away_cost_up = self.pos[blocker]+rh.length[blocker]-rh.move_on[blocked]
        if rh.move_on[blocked] + rh.length[blocker] <= 5:        #can we move it down /right?
            move_away_cost_down = 1 - self.pos[blocker]+rh.move_on[blocked]
        if move_away_cost_up == 0 and move_away_cost_down > 0:
            move_away_cost += move_away_cost_down
        elif move_away_cost_up > 0 and move_away_cost_down == 0:
            move_away_cost += move_away_cost_up
        else:
            move_away_cost += min(move_away_cost_up,move_away_cost_down)
        return move_away_cost
    
    """
    la fonction de score récompense l'état si la voiture rouge est plus proche de la sortie et le pénalise
    avec le score donnée la fonction estimee2 expliquée ci-dessus, et le pénalise s'il remet la dernière
    voiture dans sa position précédente
    """
    def score_state(self, rh):
        self.score = 1000 + 10*self.pos[0] - self.estimee2(rh)
        if self.c == self.prev.c and self.d == -self.prev.d:
            self.score -= 50
        return self.score

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [2]:
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        if state.rock != None:
            self.free_pos[state.rock[0], state.rock[1]] = False
            
    
    """
    blocking_cars: retourne la liste des voitures qui bloquent la voiture rouge et le nombre minimal de mouvements
    nécessaires pour la libèrer
    """
    def blocking_cars(self,state):
        block_list = set()
        score = 0
        for i in range(self.nbcars):
            if not self.horiz[i]:
                if (state.pos[i]+self.length[i]-1 >= self.move_on[0]) and (state.pos[i] <= self.move_on[0]):
                    if self.move_on[i] > state.pos[0]+self.length[0]-1:
                        block_list.add(i)
                        score += state.move_blocker_away_cost(rh,0,i)
        return block_list,score
    
    """
    blocking_cars_level_2: retourne la liste des voitures qui bloquent les voitures qui bloquent la voiture rouge 
    et le nombre minimal de mouvements nécessaires pour libèrer les voitures qui bloquent la voiture rouge
    """
    def blocking_cars_level_2(self,state):
        block_list,_ = self.blocking_cars(state)
        block_list_level_2 = set()
        score = 0
        for i in range(1,self.nbcars):
            for blocker in block_list:
                if self.horiz[i]:
                    if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker]) and (state.pos[i] <= self.move_on[blocker]):
                        if ((self.move_on[i] > state.pos[blocker]+self.length[blocker]-1 ) and \
                            (self.move_on[i] <= self.move_on[0]+self.length[blocker])) or \
                        ((self.move_on[i] < state.pos[blocker]) and \
                         (self.move_on[i] >= self.move_on[0] - self.length[blocker])):
                            block_list_level_2.add(i)
                            score += state.move_blocker_away_cost(rh,blocker,i) + state.pos[i]
        return block_list_level_2,score
  
    """
    blocking_cars_level_general: fonction générale utilisée dans estimee2 et expliquée ci-dessus.
    """
    def blocking_cars_level_general(self,state,depth):
        blockers = []
        length_score = 0
        block_list,score1 = self.blocking_cars(state)
        block_list_level_2,score2 = self.blocking_cars_level_2(state)
        score = score1 + score2
        vertical = True
        if block_list: 
            blockers.append(block_list)
            length_score+= depth * len(block_list)
        if block_list_level_2: 
            blockers.append(block_list_level_2)
            length_score+= (depth-1) * len(block_list_level_2)
        count = 2
        while block_list_level_2:
            block_list_level_3 = set()
            for i in range(1,self.nbcars):
                for blocker in block_list:
                    for blocker_level_2 in block_list_level_2:
                        if ( (not self.horiz[i] and vertical) or ( self.horiz[i] and  not vertical)) :
                            if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_2]) and \
                            (state.pos[i] <= self.move_on[blocker_level_2]):
                                if ((self.move_on[i] > state.pos[blocker_level_2]+self.length[blocker_level_2]-1 ) and \
                                    (self.move_on[i] <= self.move_on[blocker]+self.length[blocker_level_2])) or \
                                ((self.move_on[i] < state.pos[blocker_level_2]) and \
                                 (self.move_on[i] >= self.move_on[blocker] - self.length[blocker_level_2])):
                                    block_list_level_3.add(i)
                                    if (depth - count) >= 3: score +=  state.move_blocker_away_cost(rh,blocker,i)
            block_list = block_list_level_2
            block_list_level_2 = block_list_level_3
            vertical = not vertical
            if block_list_level_3 and len(blockers)< depth:
                blockers.append(block_list_level_3)
                length_score+= (depth-count) * len(block_list_level_3)
                count+=1
            else: 
                break
        return blockers,score,length_score
    """
    possible_moves: mouvements possibles des voitures à partir d'un état 
    """
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states
    """
    possible_rock_moves: mouvements possibles des roches à partir d'un état 
    """
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        for i in range(6):
            if i == 2: 
                continue
            for j in range(6):
                if self.free_pos[i,j] == True:
                    if state.rock == None:
                        new_states.append(state.put_rock((i,j)))
                    elif (i != state.rock[0]) and (j != state.rock[1]):
                        new_states.append(state.put_rock((i,j)))
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock != None:
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [25]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [26]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversariel avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [3]:
class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self.visited = 0

    def minimax_1(self, current_depth, current_state):
        #best_move = [current_state]
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        else:
            max_score = float('-inf')
            possible_states = self.rushhour.possible_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                s, current_score = self.minimax_1(current_depth-1, p)
                if current_score > max_score:
                    max_score = current_score
                    best_move = s
          
        #best_state = best_move[randint(0, len(best_move)-1)]
        return best_move, max_score
    
    def minimax_2(self, current_depth, current_state, is_max):
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        if is_max:
            max_score = float('-inf')
            possible_states = self.rushhour.possible_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                self.visited += 1
                _, current_score = self.minimax_2(current_depth-1, p, False)
                if current_score > max_score:
                    max_score = current_score
                    best_move_max = p
            return best_move_max, max_score
        else:
            min_score = float('inf')
            possible_states = self.rushhour.possible_rock_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                self.visited += 1
                _, current_score = self.minimax_2(current_depth-1, p, True)
                if current_score < min_score:
                    min_score = current_score
                    best_move_min = p
            return best_move_min, min_score

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        if is_max:
            max_score = float('-inf')
            possible_states = self.rushhour.possible_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                self.visited += 1
                _, current_score = self.minimax_pruning(current_depth-1, p, False, alpha, beta)
                if current_score > max_score:
                    max_score = current_score
                    best_move_max = p
                    
                alpha = max(alpha, current_score)
                if beta <= alpha: 
                    break
                    
            return best_move_max, max_score
        else:
            min_score = float('inf')
            possible_states = self.rushhour.possible_rock_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                self.visited += 1
                _, current_score = self.minimax_pruning(current_depth-1, p, True, alpha, beta)
                if current_score < min_score:
                    min_score = current_score
                    best_move_min = p
                    
                beta = min(beta, current_score)
                if beta <= alpha:
                    break
                    
            return best_move_min, min_score

    
    def expectimax(self, current_depth, current_state, is_max, fn_type):
        #fn_type: vision of the function
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        if is_max:
            max_score = float('-inf')
            possible_states = self.rushhour.possible_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                _, current_score = self.expectimax(current_depth-1, p, False, fn_type)
                if current_score > max_score:
                    max_score = current_score
                    best_move_max = p
                    
            return best_move_max, max_score
        else:
            if fn_type == "random": #random
                #random vision agent of expectimax Vs random player
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                #random.shuffle(possible_states)
                probability = 1.0 / len(possible_states)
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)

                    sum_scores += current_score * probability
                return random.choice(possible_states), sum_scores
            
            elif fn_type == "optimistic": #optimistic
                #optimistic agent of expectimax Vs random player
                max_score = float('-inf')
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                #random.shuffle(possible_states)
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)
                    if current_score > max_score:
                        max_score = current_score
                        best_move_max = p
                return random.choice(possible_states), max_score
            
            elif fn_type == "pessimistic": #pessimistic
                #pessimistic agent of expectimax Vs random player
                min_score = float('inf')
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)

                    if current_score < min_score:
                        min_score = current_score
                        best_move_min = p
                
                return random.choice(possible_states), min_score

            elif fn_type == "randomVsMaster": #random
                #agent with random vision playing Vs a Master agent
                min_score = float('inf')
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                #random.shuffle(possible_states)
                probability = 1.0 / len(possible_states)
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)
                    if current_score < min_score:
                        min_score = current_score
                        best_move_min = p
                    sum_scores += current_score * probability
                return best_move_min, sum_scores
            
            elif fn_type == "minimax": #random
                #pessimistic agent Vs Master player --> Minimax
                min_score = float('inf')
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                #random.shuffle(possible_states)
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)
                    if current_score < min_score:
                        min_score = current_score
                        best_move_min = p
                return best_move_min, min_score
            
            else:
                print("Wrong function type; please choose random, optimistic or pessimistic")
                return -1

    def decide_best_move_1(self, state):
        best_move, _ = self.minimax_1(self.search_depth, state)
        self.state = self.state.move(best_move.c, best_move.d)
        return best_move

    def decide_best_move_2(self, state, is_max):
        best_move, _ = self.minimax_2(self.search_depth, state, is_max)
        if is_max:
            self.state = self.state.move(best_move.c, best_move.d)
        else:
            self.state = self.state.put_rock(best_move.rock)
        return best_move

    def decide_best_move_pruning(self, state, is_max, alpha, beta):
        best_move, _ = self.minimax_pruning(self.search_depth, state, is_max, alpha, beta)
        if is_max:
            self.state = self.state.move(best_move.c, best_move.d)
        else:
            self.state = self.state.put_rock(best_move.rock)
        return best_move

    def decide_best_move_expectimax(self, state, is_max, fn_type):
        best_move, _ = self.expectimax(self.search_depth, state, is_max, fn_type)
        if is_max:
            self.state = self.state.move(best_move.c, best_move.d)
        else:
            self.state = self.state.put_rock(best_move.rock)
        return best_move
    
    def solve(self, state, is_singleplayer, rh):
        fifo = deque([state])
        count = 0
        if is_singleplayer:
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                if count > 50:
                    print("Nombre de mouvements > 50, veuillez relancer le code pour un meilleur résultat!")
                    return -1
                chosen_child = self.decide_best_move_1(p)
                self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                count+=1
                fifo.append(chosen_child)
        else:
            flag = True
            visited = 0
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                if count > 50:
                    print("Nombre de mouvements > 50, veuillez relancer le code pour un meilleur résultat!")
                    return -1
                chosen_child = self.decide_best_move_2(p, flag)
                #rh.print_pretty_grid(chosen_child)
                self.print_move(flag, chosen_child)
                if flag:
                    flag = False
                else:
                    flag = True
                count+=1
                fifo.append(chosen_child)
                print("Nombre de noeuds considérés = ", self.visited)
    
    def solve_pruning(self, state, is_singleplayer):
        fifo = deque([state])
        count =0
        if is_singleplayer:
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                if count > 50:
                    print("Nombre de mouvements > 50, veuillez relancer le code pour un meilleur résultat!")
                    return -1
                chosen_child = self.decide_best_move_1(p)
                self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                count+=1
                fifo.append(chosen_child)
        else:
            alpha, beta = float('-inf'), float('inf')
            flag = True
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                if count > 50:
                    print("Nombre de mouvements > 50, veuillez relancer le code pour un meilleur résultat!")
                    return -1
                chosen_child = self.decide_best_move_pruning(p, flag, alpha, beta)
                #rh.print_pretty_grid(chosen_child)
                self.print_move(flag, chosen_child)
                if flag:
                    flag = False
                else:
                    flag = True
                count+=1
                fifo.append(chosen_child)
                print("Nombre de noeuds considérés = ", self.visited)
    
    def solve_expectimax(self, state, is_singleplayer, fn_type):
        fifo = deque([state])
        count =0
        if is_singleplayer:
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                if count > 50:
                    print("Nombre de mouvements > 50, veuillez relancer le code pour un meilleur résultat!")
                    return -1
                chosen_child = self.decide_best_move_1(p)
                self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                count+=1
                fifo.append(chosen_child)
        else:
            flag = True
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                if count > 50:
                    print("Nombre de mouvements > 50, veuillez relancer le code pour un meilleur résultat!")
                    return -1
                chosen_child = self.decide_best_move_expectimax(p, flag, fn_type)
                #rh.print_pretty_grid(chosen_child)
                self.print_move(flag, chosen_child)
                if flag:
                    flag = False
                else:
                    flag = True
                count+=1
                fifo.append(chosen_child)
                
    def solve_expectimax_without_print(self, state, is_singleplayer, fn_type):
        fifo = deque([state])
        count =0
        if is_singleplayer:
            while fifo:
                p=fifo.popleft()
                if p.success():
                    #print("Nombre de mouvements: "+str(count))
                    return count
                if count > 50:
                    return -1
                chosen_child = self.decide_best_move_1(p)
                #self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                count+=1
                fifo.append(chosen_child)
        else:
            flag = True
            while fifo:
                p=fifo.popleft()
                if p.success():
                    #print("Nombre de mouvements: "+str(count))
                    return count
                if count > 50: #infinite loop
                    return -1
                chosen_child = self.decide_best_move_expectimax(p, flag, fn_type)
                #rh.print_pretty_grid(chosen_child)
                #self.print_move(flag, chosen_child)
                if flag:
                    flag = False
                else:
                    flag = True
                count+=1
                fifo.append(chosen_child)
    
    def print_move(self, is_max, state):
        if state:
            if is_max:
                msg = str(self.state.nb_moves) + ". Voiture "
                msg += self.rushhour.color[state.c] + " vers "
                if self.rushhour.horiz[state.c]:
                    if state.d == 1:
                        msg += "la droite"
                    else:
                        msg += "la gauche"
                else:
                    if state.d ==1:
                        msg += "le bas"
                    else:
                        msg += "le haut"
                print(msg)
            elif state.rock:
                msg = str(self.state.nb_moves) + ". Roche dans la case "+str(state.rock[0])+"-"+str(state.rock[1])
                print(msg)
        return None


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [95]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

2. Voiture rouge vers la droite
2. Roche dans la case 3-1


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [107]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True, rh)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture rouge vers la droite
2. Voiture bleu vers le haut
3. Voiture orange vers le haut
4. Voiture jaune vers la gauche
5. Voiture vert vers le bas
6. Voiture vert vers le bas
7. Voiture vert vers le bas
8. Voiture rouge vers la droite
9. Voiture rouge vers la droite
Nombre de mouvements: 9
Wall time: 0 ns


In [108]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True, rh) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture emeraude vers la gauche
2. Voiture lime vers la gauche
3. Voiture jaune vers le bas
4. Voiture lime vers la gauche
5. Voiture lime vers la gauche
6. Voiture bleu vers le bas
7. Voiture jaune vers le bas
8. Voiture vert vers la droite
9. Voiture mauve vers le haut
10. Voiture orange vers le haut
11. Voiture emeraude vers la gauche
12. Voiture jaune vers le bas
13. Voiture bleu vers le bas
14. Voiture rouge vers la droite
15. Voiture rouge vers la droite
16. Voiture rouge vers la droite
Nombre de mouvements: 16
Wall time: 0 ns


In [122]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True, rh)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
1. Voiture rouge vers la droite
2. Voiture 1 vers le bas
3. Voiture 2 vers la gauche
4. Voiture 2 vers la gauche
5. Voiture 2 vers la gauche
6. Voiture 3 vers le haut
7. Voiture rouge vers la droite
8. Voiture 10 vers la gauche
9. Voiture 4 vers le haut
10. Voiture 4 vers le haut
11. Voiture rouge vers la droite
12. Voiture 5 vers le bas
13. Voiture 5 vers le bas
14. Voiture rouge vers la droite
Nombre de mouvements: 14
Wall time: 0 ns


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [19]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, rh)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture bleu vers le haut
Nombre de noeuds considérés =  838
2. Roche dans la case 4-5
Nombre de noeuds considérés =  2237
3. Voiture rouge vers la droite
Nombre de noeuds considérés =  2645
4. Roche dans la case 3-0
Nombre de noeuds considérés =  3568
5. Voiture vert vers le bas
Nombre de noeuds considérés =  3946
6. Roche dans la case 1-1
Nombre de noeuds considérés =  4959
7. Voiture orange vers le haut
Nombre de noeuds considérés =  5580
8. Roche dans la case 4-5
Nombre de noeuds considérés =  7083
9. Voiture jaune vers la gauche
Nombre de noeuds considérés =  7793
10. Roche dans la case 5-4
Nombre de noeuds considérés =  9005
11. Voiture vert vers le bas
Nombre de noeuds considérés =  9570
12. Roche dans la case 1-3
Nombre de noeuds considérés =  

In [29]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, rh) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture emeraude vers la gauche
Nombre de noeuds considérés =  756
2. Roche dans la case 4-2
Nombre de noeuds considérés =  1656
3. Voiture jaune vers le bas
Nombre de noeuds considérés =  2032
4. Roche dans la case 1-4
Nombre de noeuds considérés =  2588
5. Voiture lime vers la gauche
Nombre de noeuds considérés =  2953
6. Roche dans la case 4-2
Nombre de noeuds considérés =  3576
7. Voiture vert vers la droite
Nombre de noeuds considérés =  4008
8. Roche dans la case 3-4
Nombre de noeuds considérés =  4918
9. Voiture mauve vers le haut
Nombre de noeuds considérés =  5595
10. Roche dans la case 0-3
Nombre de noeuds considérés =  6349
11. Voiture lime vers la gauche
Nombre de noeuds considérés =  7124
12. Roche dans la case 4-1
Nombre de noeuds considé

In [30]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, rh)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
1. Voiture 2 vers la gauche
Nombre de noeuds considérés =  553
2. Roche dans la case 0-1
Nombre de noeuds considérés =  1179
3. Voiture rouge vers la droite
Nombre de noeuds considérés =  1623
4. Roche dans la case 4-0
Nombre de noeuds considérés =  2076
5. Voiture 2 vers la gauche
Nombre de noeuds considérés =  2741
6. Roche dans la case 1-2
Nombre de noeuds considérés =  3204
7. Voiture 1 vers le bas
Nombre de noeuds considérés =  3803
8. Roche dans la case 0-0
Nombre de noeuds considérés =  4344
9. Voiture 10 vers la gauche
Nombre de noeuds considérés =  4820
10. Roche dans la case 1-2
Nombre de noeuds considérés =  5360
11. Voiture 2 vers la gauche
Nombre de noeuds considérés =  6116
12. Roche dans la case 0-3
Nombre de noeuds considérés =  6650
13. V

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

| function/results | Minimax2 | Minimax_pruning |
|------|------|------|
|   9 moves  | 17 (13899 explored nodes) | 17 (5286 explored nodes) |
|   16 moves  | 31 (18199 explored nodes) | 31 (4981 explored nodes) |
|   14 moves  | 27 (11986  explored nodes) | 27 (4056 explored nodes) |

In [37]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_pruning(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture bleu vers le haut
Nombre de noeuds considérés =  258
2. Roche dans la case 3-5
Nombre de noeuds considérés =  901
3. Voiture rouge vers la droite
Nombre de noeuds considérés =  1042
4. Roche dans la case 1-2
Nombre de noeuds considérés =  1429
5. Voiture orange vers le haut
Nombre de noeuds considérés =  1514
6. Roche dans la case 5-1
Nombre de noeuds considérés =  2003
7. Voiture vert vers le bas
Nombre de noeuds considérés =  2192
8. Roche dans la case 4-4
Nombre de noeuds considérés =  2891
9. Voiture jaune vers la gauche
Nombre de noeuds considérés =  3001
10. Roche dans la case 5-0
Nombre de noeuds considérés =  3533
11. Voiture vert vers le bas
Nombre de noeuds considérés =  3689
12. Roche dans la case 1-1
Nombre de noeuds considérés =  4

In [45]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_pruning(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture emeraude vers la gauche
Nombre de noeuds considérés =  289
2. Roche dans la case 3-2
Nombre de noeuds considérés =  616
3. Voiture lime vers la gauche
Nombre de noeuds considérés =  742
4. Roche dans la case 4-5
Nombre de noeuds considérés =  984
5. Voiture lime vers la gauche
Nombre de noeuds considérés =  1063
6. Roche dans la case 3-4
Nombre de noeuds considérés =  1319
7. Voiture lime vers la gauche
Nombre de noeuds considérés =  1386
8. Roche dans la case 0-2
Nombre de noeuds considérés =  1557
9. Voiture jaune vers le bas
Nombre de noeuds considérés =  1714
10. Roche dans la case 3-4
Nombre de noeuds considérés =  1985
11. Voiture bleu vers le bas
Nombre de noeuds considérés =  2086
12. Roche dans la case 4-5
Nombre de noeuds considérés =

In [53]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_pruning(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
1. Voiture rouge vers la droite
Nombre de noeuds considérés =  188
2. Roche dans la case 1-1
Nombre de noeuds considérés =  418
3. Voiture 2 vers la gauche
Nombre de noeuds considérés =  551
4. Roche dans la case 0-5
Nombre de noeuds considérés =  693
5. Voiture 2 vers la gauche
Nombre de noeuds considérés =  815
6. Roche dans la case 1-2
Nombre de noeuds considérés =  1040
7. Voiture 10 vers la gauche
Nombre de noeuds considérés =  1288
8. Roche dans la case 0-5
Nombre de noeuds considérés =  1482
9. Voiture 1 vers le bas
Nombre de noeuds considérés =  1678
10. Roche dans la case 4-1
Nombre de noeuds considérés =  1876
11. Voiture 2 vers la gauche
Nombre de noeuds considérés =  2019
12. Roche dans la case 0-3
Nombre de noeuds considérés =  2195
13. Voitu

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

##### Final results for 10 games : (Average + Min + Max)
To Double-Check it, please run the last cell of this section, it would take some time to play all the games, otherwise, there is one ouput saved in the text file "4-Expectimax - 10 Games output + Average + Min + Max.txt"

Average number of moves:

| vision 	| random 	| pessimistic 	| optimistic 	|
|----------	|--------	|-------------	|------------	|
| 9 moves  	| 19.0   	| 25.80       	| 21.8       	|
| 16 moves 	| 34.2   	| 35.00       	| 34.2       	|
| 14 moves 	| 27.6   	| 27.75       	| 27.8       	|

Minimum number of moves:

| vision 	| random 	| pessimistic 	| optimistic 	|
|----------	|--------	|-------------	|------------	|
| 9 moves  	| 17     	| 17          	| 17         	|
| 16 moves 	| 31     	| 31          	| 31         	|
| 14 moves 	| 27     	| 27          	| 27         	|

Maximum number of moves:

| vision 	| random 	| pessimistic 	| optimistic 	|
|----------	|--------	|-------------	|------------	|
| 9 moves  	| 23     	| 35          	| 29         	|
| 16 moves 	| 45     	| 45          	| 49         	|
| 14 moves 	| 29     	| 31          	| 31         	|

###### Results analysis: 
- As illustrated in the summary table above, the best average number of moves is performed by the random vision.
- We also notice that the pessimistic and optimistic visions seem to be doing worse because they either play a move "too confidently" assuming the adversary would make a bad move, or "too safely" assuming the adversary will make a great move making the game take longer.

In [36]:
# solution optimale: 9 moves

rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_expectimax(s, False, "random")
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture bleu vers le haut
2. Roche dans la case 3-2
3. Voiture rouge vers la droite
4. Roche dans la case 0-3
5. Voiture orange vers le haut
6. Roche dans la case 4-5
7. Voiture jaune vers la gauche
8. Roche dans la case 0-2
9. Voiture vert vers le bas
10. Roche dans la case 4-5
11. Voiture vert vers le bas
12. Roche dans la case 0-0
13. Voiture vert vers le bas
14. Roche dans la case 5-5
15. Voiture rouge vers la droite
16. Roche dans la case 4-3
17. Voiture rouge vers la droite
Nombre de mouvements: 17
Wall time: 0 ns


In [37]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_expectimax(s, False, "random") 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture emeraude vers la gauche
2. Roche dans la case 3-5
3. Voiture lime vers la gauche
4. Roche dans la case 4-2
5. Voiture jaune vers le bas
6. Roche dans la case 1-1
7. Voiture lime vers la gauche
8. Roche dans la case 5-4
9. Voiture lime vers la gauche
10. Roche dans la case 4-3
11. Voiture jaune vers le bas
12. Roche dans la case 0-4
13. Voiture bleu vers le bas
14. Roche dans la case 1-2
15. Voiture jaune vers le bas
16. Roche dans la case 0-3
17. Voiture jaune vers le haut
18. Roche dans la case 4-4
19. Voiture vert vers la droite
20. Roche dans la case 3-1
21. Voiture mauve vers le haut
22. Roche dans la case 0-4
23. Voiture orange vers le haut
24. Roche dans la case 5-5
25. Voiture emeraude vers la gauche
26. Roche dans la case 3-2
27. Voitur

In [38]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_expectimax(s, False, "random")
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
1. Voiture rouge vers la droite
2. Roche dans la case 1-4
3. Voiture 2 vers la gauche
4. Roche dans la case 0-1
5. Voiture 1 vers le bas
6. Roche dans la case 4-3
7. Voiture 2 vers la gauche
8. Roche dans la case 1-1
9. Voiture 2 vers la gauche
10. Roche dans la case 0-3
11. Voiture 10 vers la gauche
12. Roche dans la case 4-1
13. Voiture 3 vers le haut
14. Roche dans la case 0-5
15. Voiture 4 vers le haut
16. Roche dans la case 1-1
17. Voiture 4 vers le haut
18. Roche dans la case 0-5
19. Voiture rouge vers la droite
20. Roche dans la case 4-0
21. Voiture rouge vers la droite
22. Roche dans la case 0-5
23. Voiture 5 vers le bas
24. Roche dans la case 1-2
25. Voiture 5 vers le bas
26. Roche dans la case 0-5
27. Voiture rouge vers la droite
Nombre de mouve

In [133]:
import pandas as pd
import sys
# solutions optimales respectivement: 9 moves, 16 moves, 14 moves
rh0 = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
rh1 = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
rh2 = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
rhs = [rh0 , rh1 , rh2 ]
s0 = State([1, 0, 1, 3, 2])
s1 = State([1, 0, 1, 4, 2, 4, 0, 1])
s2 = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
ss = [s0, s1, s2]

vision = ["random","pessimistic","optimistic"] # all of games are Vs random player

results = np.zeros((3,3),dtype=int)

number_of_games = 10

final_results = np.zeros((3,3,number_of_games),dtype=int)
avg_results = np.zeros((3,3),dtype=float)
min_results = np.zeros((3,3),dtype=int)
max_results = np.zeros((3,3),dtype=int)

print("Note 1: If the number of moves is more than 50, we put -1 instead!")
print("Note 2: Please be patient while the games are being played, and no worries about infinite loops, they are handled!\n")

for game in range(number_of_games):
    count = 1
    j=0
    print("\nGame :",game+1,"/",number_of_games)
    for v in vision:
        for i in range(3):
            algo = MiniMaxSearch(rhs[i], ss[i],3)
            algo.rushhour.init_positions(ss[i])
            results[i][j] = algo.solve_expectimax_without_print(ss[i], False, v)
            final_results[i][j][game] = results[i][j]
            sys.stdout.write('\r')
            # just a fancy progress bar:
            sys.stdout.write("[%-18s] %.1f%%" % ('='*2*count, (100/9)*count))
            sys.stdout.flush()
            count +=1
        j+=1
    print()
    tableau = pd.DataFrame(results,columns = ["random","pessimistic","optimistic"],
                                   index = ["9 moves","16 moves","14 moves"])
    print(tableau)
print("\n\nFinal results of all games : (Average + Min + Max)")

for i in range(3):
    for j in range(3):
        avg_results[i][j] = round(np.average([x for x in final_results[i][j] if x != -1]),2)
        min_results[i][j] = np.min([x for x in final_results[i][j] if x != -1])
        max_results[i][j] = np.max([x for x in final_results[i][j] if x != -1])
        
print("\nAverage number of moves:")
print(pd.DataFrame(avg_results,columns = ["random","pessimistic","optimistic"],
                                   index = ["9 moves","16 moves","14 moves"]))        
print("\nMinimum number of moves:")
print(pd.DataFrame(min_results,columns = ["random","pessimistic","optimistic"],
                                   index = ["9 moves","16 moves","14 moves"]))        
print("\nMaximum number of moves:")
print(pd.DataFrame(max_results,columns = ["random","pessimistic","optimistic"],
                                   index = ["9 moves","16 moves","14 moves"]))

%time

Note 1: If the number of moves is more than 50, we put -1 instead!
Note 2: Please be patient while the games are being played, and no worries about infinite loops, they are handled!


Game : 1 / 10
[==================] 100.0%
          random  pessimistic  optimistic
9 moves       19           33          -1
16 moves      31           31          31
14 moves      27           31          -1

Game : 2 / 10
[==================] 100.0%
          random  pessimistic  optimistic
9 moves       17           35          37
16 moves      31           31          43
14 moves      27           29          -1

Game : 3 / 10
[==================] 100.0%
          random  pessimistic  optimistic
9 moves       21           31          19
16 moves      31           39          37
14 moves      27           31          31

Game : 4 / 10
[==================] 100.0%
          random  pessimistic  optimistic
9 moves       21           27          39
16 moves      31           35          37
14 moves      2

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 

#### Victoires/defaites:

| Players/Results | Minimax vs Adversaire optimal | Minimax vs Adversaire random | Expectimax vs Adversaire optimal | Expectimax vs Adversaire random |
|------|------|------|------|------|
| avg nbr of moves for 9 moves | 28.2 | 24.2 | 17.6 | 20.0 |
| avg nbr of moves for 16 moves | 42.4 | 34.0 | 37.0 | 33.8 |
| avg nbr of moves for 14 moves | 31.8 | 32.0 | 28.0 | 29.0 |
|------|------|------|------|------|
| max nbr of moves for 9 moves | 47 | 31 | 19 | 29 |
| max nbr of moves for 16 moves | 49 | 39 | 39 | 39 |
| max nbr of moves for 14 moves | 45 | 39 | 29 | 37 |
|------|------|------|------|------|
| min nbr of moves for 9 moves | 21 | 17 | 17 | 17 |
| min nbr of moves for 16 moves | 35 | 31 | 35 | 31 |
| min nbr of moves for 14 moves | 27 | 27 | 27 | 27 |
|------|------|------|------|------|
| nbr of wins for 9 moves | 7 | 4 | 7 | 6 |
| nbr of wins for 16 moves | 4 | 4 | 5 | 5 |
| nbr of wins for 14 moves | 7 | 4 | 5 | 7 |

###### Notes: 
- This table is a summary of the output of the last cell of this notebook ( feel free to run it again to verify (: )
- Threshold for win/loss of a game is the average value of the number of moves in each game
- Search depth = 3 is the same for all players

###### Results analysis: 
- As illustrated in the summary table above, the best player in terms of number of wins is Minimax vs Adversaire optimal.
- We also notice that Minimax vs random and Expectimax vs optimal seem to be doing worse because the player is making a wrong prediction about the adversary which either plays a move randomly or cautiously making the game take longer.
- On the other hand, Minimax vs optimal and Expectimax vs random are performing better because they are making reasonable predictions about the adversary's moves.

In [143]:
import pandas as pd
import sys
# solutions optimales respectivement: 9 moves, 16 moves, 14 moves
rh0 = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
rh1 = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
rh2 = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
rhs = [rh0 , rh1 , rh2 ]
s0 = State([1, 0, 1, 3, 2])
s1 = State([1, 0, 1, 4, 2, 4, 0, 1])
s2 = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
ss = [s0, s1, s2]

vision = ["minimax","randomVsMaster","pessimistic","random"] # 2 games Vs Master player, 2 games Vs random player

results = np.zeros((3,4),dtype=int)

number_of_games = 10

final_results = np.zeros((3,4,number_of_games),dtype=int)
avg_results = np.zeros((3,4),dtype=float)
min_results = np.zeros((3,4),dtype=int)
max_results = np.zeros((3,4),dtype=int)
win_results = np.zeros((3,4),dtype=int)

print("Note 1: If the number of moves is more than 50, we put -1 instead!")
print("Note 2: Please be patient while the games are being played, and no worries about infinite loops, they are handled!\n")

for game in range(number_of_games):
    count = 1
    j=0
    print("\nGame :",game+1,"/",number_of_games)
    for v in vision:
        for i in range(3):
            algo = MiniMaxSearch(rhs[i], ss[i],3)
            algo.rushhour.init_positions(ss[i])
            results[i][j] = algo.solve_expectimax_without_print(ss[i], False, v)
            final_results[i][j][game] = results[i][j]
            sys.stdout.write('\r')
            # just a fancy progress bar:
            sys.stdout.write("[%-24s] %.1f%%" % ('='*2*count, (100/12)*count))
            sys.stdout.flush()
            count +=1
        j+=1
    print() 
    tableau = pd.DataFrame(results,columns = ["minimax Vs optimal","expectimax neutral Vs optimal","minimax Vs random","expectimax neutral Vs random"],
                                   index = ["9 moves","16 moves","14 moves"])
    print(tableau)

print("\n\nFinal results of all games : (Average + Min + Max)")

for i in range(3):
    for j in range(4):
        avg_results[i][j] = round(np.average([x for x in final_results[i][j] if x != -1]),2)
        min_results[i][j] = np.min([x for x in final_results[i][j] if x != -1])
        max_results[i][j] = np.max([x for x in final_results[i][j] if x != -1])
        win_results[i][j] = len([x for x in final_results[i][j] if x != -1 and x <= round(avg_results[i][j])])
        
print("\nAverage number of moves:")
print(pd.DataFrame(avg_results,columns = ["minimax Vs optimal","expectimax neutral Vs optimal","minimax Vs random","expectimax neutral Vs random"],
                                   index = ["9 moves","16 moves","14 moves"]))        
print("\nMinimum number of moves:")
print(pd.DataFrame(min_results,columns = ["minimax Vs optimal","expectimax neutral Vs optimal","minimax Vs random","expectimax neutral Vs random"],
                                   index = ["9 moves","16 moves","14 moves"]))        
print("\nMaximum number of moves:")
print(pd.DataFrame(max_results,columns = ["minimax Vs optimal","expectimax neutral Vs optimal","minimax Vs random","expectimax neutral Vs random"],
                                   index = ["9 moves","16 moves","14 moves"]))        
print("\nNumber of Wins with threshold of win is the average value of number of moves in each game")
print(pd.DataFrame(win_results,columns = ["minimax Vs optimal","expectimax neutral Vs optimal","minimax Vs random","expectimax neutral Vs random"],
                                   index = ["9 moves","16 moves","14 moves"]))

%time

Note 1: If the number of moves is more than 50, we put -1 instead!
Note 2: Please be patient while the games are being played, and no worries about infinite loops, they are handled!


Game : 1 / 10
[========================] 100.0%
          minimax Vs optimal  expectimax neutral Vs optimal  \
9 moves                   23                             17   
16 moves                  37                             35   
14 moves                  27                             29   

          minimax Vs random  expectimax neutral Vs random  
9 moves                  29                            17  
16 moves                 39                            31  
14 moves                 -1                            29  

Game : 2 / 10
[========================] 100.0%
          minimax Vs optimal  expectimax neutral Vs optimal  \
9 moves                   25                             19   
16 moves                  43                             35   
14 moves                  45         